In [1]:
import os
from src.evaluation.save_results import create_output_file, save_results

output_file_name = "chronos_bolt_mini_test_results1.csv"

output_dir="results/test"
csv_file_path = os.path.join(output_dir, output_file_name)

create_output_file(output_file_name, output_dir)


In [2]:
from gluonts.model import evaluate_model
from gluonts.time_feature import get_seasonality
from src.evaluation.load_chronos_data import load_data

In [3]:
from src.evaluation.metrics import get_metrics

# Instantiate the metrics
metrics = get_metrics()

### Data

In [4]:
import json
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

short_datasets = "solar/10T solar/H solar/D solar/W jena_weather/10T jena_weather/H jena_weather/D " \
"bitbrains_fast_storage/5T bitbrains_fast_storage/H bitbrains_rnd/5T bitbrains_rnd/H bizitobs_application " \
"bizitobs_service bizitobs_l2c/5T bizitobs_l2c/H"

med_long_datasets = "solar/10T solar/H jena_weather/10T jena_weather/H " \
"bitbrains_fast_storage/5T bitbrains_rnd/5T bizitobs_application bizitobs_service bizitobs_l2c/5T bizitobs_l2c/H"

# Get union of short and med_long datasets
all_datasets = list(set(short_datasets.split() + med_long_datasets.split()))

dataset_properties_map = json.load(open("data/dataset_properties.json"))

In [5]:
from src.evaluation.load_data import load_gift_data

load_gift_data()

/home/valerio/miniconda3/envs/chronos2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/valerio/miniconda3/envs/chronos2/lib/python3.9/site-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 175 files:   0%|                                                                                                           | 0/175 [00:00<?, ?it/s]

Fetching 175 files:  79%|██████████████████████████████████████████████████████████████████████████▉                    | 138/175 [00:00<00:00, 1372.29it/s]

Fetching 175 files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 175/175 [00:00<00:00, 1453.65it/s]

Available datasets in GIFT_EVAL:
- LOOP_SEATTLE/5T
- LOOP_SEATTLE/D
- LOOP_SEATTLE/H
- M_DENSE/D
- M_DENSE/H
- SZ_TAXI/15T
- SZ_TAXI/H
- bitbrains_fast_storage/5T
- bitbrains_fast_storage/H
- bitbrains_rnd/5T
- bitbrains_rnd/H
- bizitobs_application
- bizitobs_l2c/5T
- bizitobs_l2c/H
- bizitobs_service
- car_parts_with_missing
- covid_deaths
- electricity/15T
- electricity/D
- electricity/H
- electricity/W
- ett1/15T
- ett1/D
- ett1/H
- ett1/W
- ett2/15T
- ett2/D
- ett2/H
- ett2/W
- hierarchical_sales/D
- hierarchical_sales/W
- hospital
- jena_weather/10T
- jena_weather/D
- jena_weather/H
- kdd_cup_2018_with_missing/D
- kdd_cup_2018_with_missing/H
- m4_daily
- m4_hourly
- m4_monthly
- m4_quarterly
- m4_weekly
- m4_yearly
- restaurant
- saugeenday/D
- saugeenday/M
- saugeenday/W
- solar/10T
- solar/D
- solar/H
- solar/W
- temperature_rain_with_missing
- us_births/D
- us_births/M
- us_births/W


In [6]:
DATASET_NAME = [
    'bizitobs_l2c/5T',
    'bitbrains_fast_storage/5T',
    'bitbrains_rnd/H',
    'bizitobs_l2c/H',
    'jena_weather/H',
    'bizitobs_application',
    'bizitobs_service',
    'bitbrains_fast_storage/H',
    'solar/D',
    'jena_weather/10T',
    'solar/W',
    'jena_weather/D',
    'bitbrains_rnd/5T',
    'solar/10T',
    'solar/H',
]

In [7]:
CHRONOS_DATASET_NAME = ["exchange_rate", "ercot", "dominick"]

### Evaluation

In [8]:
MODEL_PATH = "/raid/decaro/TimeSeriesForecastingFoundationModels/chronos_output/chronos-bolt-mini/run-1/"
MODEL_NAME = "chronos-bolt-mini"

#CHECKPOINTS = ["checkpoint-500"] + [f"checkpoint-{i*10000}" for i in range(1, 4)] # range(1, 20+1) 70000
CHECKPOINTS = [f"checkpoint-{i*10000}" for i in range(8, 20+1)]

In [9]:
import sys
sys.path.insert(0, "src")

In [10]:
from src.evaluation.chronos_predictor import ChronosPredictor
from src.gift_eval.data import Dataset

for model_name in CHECKPOINTS:
    train_step = int(model_name.split("-")[-1])
    
    for ds_name in CHRONOS_DATASET_NAME:
        print(f"Processing dataset: {ds_name}")

        terms = ["short", "medium", "long"]
        for term in terms:
            ds_config = f"{ds_name}/{term}"

            dataset, prediction_length, frequency, domain, num_variates = load_data(ds_name, term)
            season_length = get_seasonality(frequency)

            predictor = ChronosPredictor(
                model_path=MODEL_PATH+model_name,
                num_samples=20,
                prediction_length=prediction_length,
                device_map="cpu",
            )

            res = evaluate_model(
                predictor,
                test_data=dataset,
                metrics=metrics,
                batch_size=512,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            save_results(res, ds_config, MODEL_NAME, train_step, domain, num_variates, ds_name, csv_file_path)

    for ds_name in DATASET_NAME:
        ds_key = ds_name.split("/")[0]
        print(f"Processing dataset: {ds_name}")
        terms = ["short", "medium", "long"]
        for term in terms:
            if (
                term == "medium" or term == "long"
            ) and ds_name not in med_long_datasets.split():
                continue

            if "/" in ds_name:
                ds_key = ds_name.split("/")[0]
                ds_freq = ds_name.split("/")[1]
                ds_key = ds_key.lower()
            else:
                ds_key = ds_name.lower()
                ds_freq = dataset_properties_map[ds_key]["frequency"]
            ds_config = f"{ds_key}/{ds_freq}/{term}"

            # Initialize the dataset
            to_univariate = (
                False
                if Dataset(name=ds_name, term=term, to_univariate=False).target_dim == 1
                else True
            )
            dataset = Dataset(name=ds_name, term=term, to_univariate=to_univariate)
            season_length = get_seasonality(dataset.freq)

            predictor = ChronosPredictor(
                model_path=MODEL_PATH+model_name,
                num_samples=20,
                prediction_length=dataset.prediction_length,
                device_map="cpu",
            )

            # Measure the time taken for evaluation
            res = evaluate_model(
                predictor,
                test_data=dataset.test_data,
                metrics=metrics,
                batch_size=1024,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            save_results(res, ds_config, MODEL_NAME, train_step, domain, num_variates, ds_name, csv_file_path)


Unknown architecture: T5ForConditionalGeneration, defaulting to ChronosBoltModelForForecasting


Processing dataset: exchange_rate
prediction_length: 8


Some weights of ChronosBoltModelForForecasting were not initialized from the model checkpoint at /raid/decaro/TimeSeriesForecastingFoundationModels/chronos_output/chronos-bolt-mini/run-1/checkpoint-80000 and are newly initialized because the shapes did not match:
- shared.weight: found shape torch.Size([4096, 384]) in the checkpoint and torch.Size([1, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0it [00:00, ?it/s]

1it [00:04,  4.75s/it]

...

63it [12:53, 12.47s/it]

64it [13:05, 12.42s/it]

65it [13:17, 12.33s/it]

66it [13:28, 11.80s/it]

67it [13:39, 11.58s/it]